## Сшивание изображений ##

Так как задача в целом весьма стандартная, собирать будем из стандартных же кусков. За основу взят алгоритм статьи: [Brown, Lowe. Automatic Panoramic Image Stitching using Invariant Features](http://matthewalunbrown.com/papers/ijcv2007.pdf) По сути, большинство найденных мной по этой теме текстов все равно ссылаются на ту работу и прелагают лишь небольшие улучшения.

Краткий эмпирический (глазами посмотрел :) ) анализ входных данных позволил предположить следующее:
1. все изображения каждого набора лежат на одной плоскости (так как нет видимых перспективных искажений узнаваемых объектов aka крупных пятен), а значит совмещаются аффинным преобразованием.
2. В первом наборе данных экспозиция всех изображений одинаковая, во втором потребуется коррекция (гамма или гейн, будем смотреть в процессе; логарифмическая мало вероятна, но будем держать в голове)
3. в условиях обещано последовательное наложение. Этим можно существенно воспользоваться, совмещая ключевые точки только соседних изображений. Такой подход в боевой ситуации неизбежно приведет к инерционной ошибке, но на синтетических данных может сработать. Если не сработает, можно попробовать easy fix: совмещать два последние изображения как начальное приближение, находить еще одно-два изображения с максимальным геометрическим пересечением с новым кусочком (назовем из вспомогательными) и повторить matching ключевых точек предпоследнего+вспомогательного изображения с последним. Такая поправка должна хорошо сработать, если изображение нарезано "змейкой" и хуже, если спиралью. В боевом применении стоит вместо этого использовать bundle adjustment из статьи выше. На синтетических данных есть надежда, что даже матчинг с последним изображением будет работать хорошо. Опять же, вспоминаем пункт 1: линейные преобразования должны давать более-менее линейную же ошибку округлений. 

### Основные шаги алгоритма: ###
1. поиск ключевых точек посредством DoG
2. построение SIFT-дескрипторов. SURF должен быть быстрее, но в python-opencv версии 5.5, которую я использовал, его еще нет (только в opencv-contib). Скорость работы SIFT оказалась приемлемой и я не стал чинить то, что работает.
3. Совмещение инвариантных представлений осуществляется при помощи FLANN. 
4. Далее, найденные совмещения фильтруются, чтобы исключить далекие пары (FLANN находит ближайшего соседа, даже если он не близок...) и точки вне пересеения (RANSAC) и по ним строится приближенное проективное преобразование (на самом деле аффинное).
5. В работе [Brown-Lowe] коррекцию экспозиции (gain) предлагается производить, минимизируя квадратичный функционал на перекрывающихся кусках. Мы упрсотим эту схему, и будем приводить гейн каждого последующего изобржения к предыдущему посредством вычисления поканального среднего отношения пикселей на пересекающихся участках. Гамма-коррекцию при этом проводить не будем (не понадобилась). Если первое изображение в серии было недосвечено, такая операция недосвечивает все последующие, поэтому добавлена возможность глобальной коррекции: если ее включить, то все коэффициенты коррекции гейна сохранаются и в финале усредняются, после чего к итоговому изобажению применяется коррекция на обратную величину.
6. К новому куску применяется гомография и он вклеивается к предыдущим

Пункты 1-4 реализуются стандартно при помощи opencv 5.5, большинство параметров взято либо по умолчанию, либо со stackoverflow. Пункт 6 требует задуматься об ориентации накопленного и добавляемого изображений на подложке. Я вижу здесь несколько подходов:
1. вращать (здесь и далее под вращением подразумевается произвольная гомография) накопленное изображение вокруг нового, тогда новое можно дописывать в левый верхний угол, а размер подложки можно вычислить, посчитав координаты углов bounding box накопленного изображения. Здесь есть два существенных минуса: а) повторяющиеся вращения одного и того же изображения приведут к накоплению интерполяционных ошибок. б) вращение большого накопленного изображения ресурсозатратно.
2. вращать новый кусочек вокруг накопленного изображения. Здесь основная проблема в том, что в координатах накопленного изображения образ нового куска может быть далеко и совмещать два больших изображения, что не только не очень быстро, но и требует, условно, удвоения используемой памяти. Я решаю эту проблему так: из гомографии выделяется трансляция, после чего заменяется на малую трансляцию, достаточную для расширения подложки, гарантирующее, что в нее поместится результат любого вращения, затем применяется получившаяся новая гомография и результат
вклеивается alpha-blending'ом с учетом необходимого смещения.

### Допущения и попущения ###

В целях экономии времени я постулирую координаты левого верхнего угла первого изображения как таковые для результата, а его стороны как направляющие сторон результата. Более того, я вручную выбираю размер подложки итогового изображения так, чтобы "влезло". Из-за этого некоторая часть изображения обрезается (слева) и остается свободное прозрачное место (справа и снизу). В рамках тестового задания мне не кажется это большим грехом, т.к. поправить это очень легко: достаточно перед вклейкой очередного изображения вычислить координаты образа его центра и, если они окажутся отрицательными, сместить накопленное изображение на подложке вправо и вниз. В конце нужно найти наименьший прямоугольник (возможно, развернутый), содержащий результат (маска результата содержится в alpha-канале) и развернуть его так, чтобы одна из сторон стала горизонтальной, после чего обрезать все за пределами прямоугольника. Также, я не стал проводить коррекцию инерционной ошибки (предложение, как это сделать, есть выше, но результат кажется мне достаточно хорошим и без этого).

Аналогично, вместо минимальной gain-коррекции по всем изображениям, каждое последующее
приводится к gain-уровню накопленного.

По той же причине я не занимаюсь аккуратным логированием и обработкой исключений.

### Артефакты и метрики ###
Для борьбы с арфектами в статье предложен алгортм Multiband Blending, но реализовать его, признаться, у меня не хватило, ни временеи ни сил. Кроме того, от интерполяции при поворотах на границе изображения появлятся ошибки, что выражается в наличии темных полос шириной в 2 пикселя. Бороться с этим я предлагаю обнулением всех 4 каналов на тонкой рамке нового изображения перед поворотом. TODO: ???????

В качестве оценки сшивания я предлагаю использовать MSE разностей пикселей в каждой области пересечения.

TODO: вычислить

N.B.: проблема с белым цветом, т.к. при пересвете он уперся в потолок, а затемняя мы его ломаем

### Реализация в opencv ###
В opencv реализован end-to-end алгоритм такого сорта -- cv2.Stitcher, но во-первых это было бы совсем уж неспортивно, а во-вторых, он не справляется: на малом количестве изображений он выдает кривулю, так как то ли счиает изображения панорамой с камеры и пытется их выпрямить, игнорируя наклоны, то ли пытается компенсировать бочкообразные искажения. А начиная с некоторого количество изоражений выжирает прорву памяти и люто тормозит. Я оставлял его на ночь с полной пачкой изображений из первого набора и он в какой-то момент просто упал. Так что точно придется собирать руками.


In [5]:
import data4
import cv2
from timeit import default_timer as timer
import stitcher

In [6]:
data_dir = "../04_image_matching/imgage-matching-samples-v2"
# img_csv = "O00401574_010_HE_clip2p1.tif-crops-c0-idx.txt"
img_csv = "O00401574_010_HE_clip3.tif-crops-c1-idx.txt"

images = data4.StitchingDataset(data_dir, img_csv)

In [7]:
'''
    Warning: when num_images is set to high values takes A LOT of time to execute.
    On my machine setting num_images = len(images) (66 for the first dataset),
    this code couldn't complete in 1 whole hour. The time complexity is nonlinear
    10 images are precessed in 25 seconds
    20 images are precessed in 245 seconds
    Thus, the time complexity is more than cubic in terms 
    number of input images (of the same size)!

    Here and after, my hardware setup is:
    i5-7300 4@2.5Ghz
    8GB RAM
    HDD
    GTX1050 2Gb (not used by cv2 though...)
'''
# cv2stitcher = cv2.Stitcher.create() #uses opencv 5.5, previous versions had differen signatures lie createStitcher or Stitcher_create
# start = timer()
# num_images = 5#len(images)
# rgb_images = [img[:,:,:3] for img in images[0:num_images]]
# status, cv2result = cv2stitcher.stitch(rgb_images)
# end = timer()
# print(f"Stitched in: {end-start} seconds")
# if not status:
#     cv2.imwrite(f"out4/deflt_stitcher_result.png", cv2result)
# else:
#     print(f"Can't stich. Status code: {status}")

"\n    Warning: when num_images is set to high values takes A LOT of time to execute.\n    On my machine setting num_images = len(images) (66 for the first dataset),\n    this code couldn't complete in 1 whole hour. The time complexity is nonlinear\n    10 images are precessed in 25 seconds\n    20 images are precessed in 245 seconds\n    Thus, the time complexity is more than cubic in terms \n    number of input images (of the same size)!\n\n    Here and after, my hardware setup is:\n    i5-7300 4@2.5Ghz\n    8GB RAM\n    HDD\n    GTX1050 2Gb (not used by cv2 though...)\n"

In [11]:
#Now to a custom stitcher
SIFT_limit = 10000 #limits the number of generated features to n best ones
mystitcher = stitcher.Stitcher(SIFT_limit)

start = timer()
num_images = 2#len(images)
target_size = (15000, 10000)
myresult = mystitcher.stitch(images[:num_images], target_size, write_intermediate=False,
    output_filename="o/stitched", 
    gain_correction = stitcher.GainCorrection.LOCAL_AND_GLOBAL)
end = timer()
print(f"Stitched in: {end-start} seconds")

Stitching: 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]


Average gain correction:  [[[1.1070898 1.1545593 1.1039008]]]
Stitched in: 18.04079200000001 seconds


### Полетные данные ###